In [133]:
import torch
import torch.nn.functional as F
import pandas as pd

from torch import nn, optim


# Prepare the data

train_file = './train.csv'
test_file = './test.csv'

def prepare_data(data, test_sample=False):
    persons = data[['PassengerId','Name','Age','Sex']]
    data = pd.get_dummies(data.drop(columns=['Cabin','Ticket','Pclass']), columns=['Sex', 'Embarked']).fillna(0)
    
    features = data[['Age', 'SibSp', 'Parch', 'Fare', 'Sex_female','Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S']].copy()
    features['Fare'] /= features['Fare'].max()
    features['Age'] /= features['Age'].max()
    features['Fare'] /= features['Fare'].max()
    features['SibSp'] /= features['SibSp'].max()
    
    features = torch.tensor(features.values, dtype=torch.float)
    persons = persons.values
    
    if(test_sample):
        return zip(features, persons)
    
    targets = data['Survived']
    targets = torch.tensor(targets.values, dtype=torch.int).view(-1,1)
    
    return zip(features, persons, targets)


In [134]:
train_data = prepare_data(pd.read_csv(train_file))
test_data = prepare_data(pd.read_csv(test_file), True)

train_f, train_person, train_t = list(train_data)[0]
test_f, test_person = list(test_data)[0]

print(train_f, test_f)

tensor([0.2750, 0.1250, 0.0000, 0.0142, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000]) tensor([0.4539, 0.0000, 0.0000, 0.0153, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000])


In [141]:
class Classifier(nn.Module):
    
    def __init__(self):
        super(Classifier, self).__init__()
        
        self.fc1 = nn.Linear(9, 6)
        self.fc2 = nn.Linear(6, 3)
        self.fc3 = nn.Linear(3, 2)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, X):
        X = self.dropout( F.relu( self.fc1(X) ) )
        X = self.dropout( F.relu( self.fc2(X) ) )
        X = self.dropout( F.softmax( self.fc3(X), dim=0 ) )
        
        return X
    

In [142]:
classifier = Classifier()
logits = classifier(train_f)
print(logits, train_t)

tensor([0.3906, 0.8594], grad_fn=<MulBackward0>) tensor([0], dtype=torch.int32)


In [ ]:
lr = 0.005

def train(target, person_tensor):
    """
        Training the data set
        Params:
            category_tensor: the expected category result
            line_tensor: the line to training
    """
    
    classifier.zero_grad()
    
    output = classifier(person_tensor)

    print(output, target)
    loss = criterion(output, category_tensor)
    loss.backward()
    
    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(-lr, p.grad.data)
    
    return output, loss.item()